# Settings & Configutations

In [ ]:
from google.cloud import storage
import os

# Models
from transformers import Wav2Vec2Model

# Google Storage

In [ ]:
client = storage.Client(project='chulaworks')
bucket = client.bucket('dwn-chula')
blobs = bucket.list_blobs(prefix='datasets/ICNALE/SM')

In [ ]:
# Overview the files in the bucket

extensions = dict()
duped_count = 0

for blob in blobs:
    blob_name = blob.name
    # extension
    name, ext = os.path.splitext(blob_name)
    if ext not in extensions:
        extensions[ext] = 0
    extensions[ext] += 1
    # duped files
    if name[-1] == ")":
        duped_count += 1

for ext, count in extensions.items():
    print(f"{ext}\t:\t{count}")
print("----------------------------------------")
print(f"Total duped files\t:\t{duped_count}")

# Preprocessing

In [ ]:
# Download files from the bucket

def download_blobs(bucket, prefix, destination_folder):
    """
    Download all blobs with the given prefix from the bucket to the destination folder.
    """
    blobs = bucket.list_blobs(prefix=prefix)
    os.makedirs(destination_folder, exist_ok=True)
    for blob in blobs:
        local_path = os.path.join(destination_folder, os.path.relpath(blob.name, prefix))
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        blob.download_to_filename(local_path)
        print(f"Downloaded {blob.name} to {local_path}")

# Pipeline

In [ ]:
model_wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

# Evaluation